# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 8 2022 12:17PM,245966,22,626816,"NBTY Global, Inc.",Released
1,Sep 8 2022 12:16PM,245965,22,626675,"NBTY Global, Inc.",Released
2,Sep 8 2022 12:15PM,245964,22,626518,"NBTY Global, Inc.",Released
3,Sep 8 2022 12:11PM,245963,10,0085968928,ISDIN Corporation,Released
4,Sep 8 2022 12:11PM,245963,10,0085968935,ISDIN Corporation,Released
5,Sep 8 2022 12:11PM,245963,10,0085969023,ISDIN Corporation,Released
6,Sep 8 2022 12:11PM,245963,10,0085969024,ISDIN Corporation,Released
7,Sep 8 2022 12:11PM,245963,10,0085969499,ISDIN Corporation,Released
8,Sep 8 2022 12:11PM,245963,10,0085969836,ISDIN Corporation,Released
9,Sep 8 2022 12:11PM,245963,10,0085969837,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,245962,Released,2
34,245963,Released,12
35,245964,Released,1
36,245965,Released,1
37,245966,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
245962,NaN,NaN,2.0
245963,NaN,NaN,12.0
245964,NaN,NaN,1.0
245965,NaN,NaN,1.0
245966,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
245650,0.0,0.0,1.0
245758,1.0,0.0,0.0
245795,0.0,0.0,1.0
245843,0.0,0.0,2.0
245889,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
245650,0.0,0,1
245758,1.0,0,0
245795,0.0,0,1
245843,0.0,0,2
245889,0.0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,245650,0.0,0,1
1,245758,1.0,0,0
2,245795,0.0,0,1
3,245843,0.0,0,2
4,245889,0.0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,245650,0.0,,1
1,245758,1.0,,
2,245795,0.0,,1
3,245843,0.0,,2
4,245889,0.0,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 8 2022 12:17PM,245966,22,"NBTY Global, Inc."
1,Sep 8 2022 12:16PM,245965,22,"NBTY Global, Inc."
2,Sep 8 2022 12:15PM,245964,22,"NBTY Global, Inc."
3,Sep 8 2022 12:11PM,245963,10,ISDIN Corporation
15,Sep 8 2022 12:11PM,245959,10,ISDIN Corporation
28,Sep 8 2022 12:08PM,245962,19,"GUSA Granules USA, Inc."
30,Sep 8 2022 11:34AM,245958,21,"NBTY Global, Inc."
31,Sep 8 2022 11:25AM,245957,19,"Graystone, LLC"
35,Sep 8 2022 11:16AM,245954,10,Bio-PRF
40,Sep 8 2022 11:10AM,245956,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Sep 8 2022 12:17PM,245966,22,"NBTY Global, Inc.",0.0,,1
1,Sep 8 2022 12:16PM,245965,22,"NBTY Global, Inc.",0.0,,1
2,Sep 8 2022 12:15PM,245964,22,"NBTY Global, Inc.",0.0,,1
3,Sep 8 2022 12:11PM,245963,10,ISDIN Corporation,0.0,,12
4,Sep 8 2022 12:11PM,245959,10,ISDIN Corporation,0.0,,13
5,Sep 8 2022 12:08PM,245962,19,"GUSA Granules USA, Inc.",0.0,,2
6,Sep 8 2022 11:34AM,245958,21,"NBTY Global, Inc.",0.0,,1
7,Sep 8 2022 11:25AM,245957,19,"Graystone, LLC",0.0,4,
8,Sep 8 2022 11:16AM,245954,10,Bio-PRF,0.0,,5
9,Sep 8 2022 11:10AM,245956,21,"NBTY Global, Inc.",0.0,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 12:17PM,245966,22,"NBTY Global, Inc.",1,
1,Sep 8 2022 12:16PM,245965,22,"NBTY Global, Inc.",1,
2,Sep 8 2022 12:15PM,245964,22,"NBTY Global, Inc.",1,
3,Sep 8 2022 12:11PM,245963,10,ISDIN Corporation,12,
4,Sep 8 2022 12:11PM,245959,10,ISDIN Corporation,13,
5,Sep 8 2022 12:08PM,245962,19,"GUSA Granules USA, Inc.",2,
6,Sep 8 2022 11:34AM,245958,21,"NBTY Global, Inc.",1,
7,Sep 8 2022 11:25AM,245957,19,"Graystone, LLC",,4
8,Sep 8 2022 11:16AM,245954,10,Bio-PRF,5,
9,Sep 8 2022 11:10AM,245956,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 12:17PM,245966,22,"NBTY Global, Inc.",1,
1,Sep 8 2022 12:16PM,245965,22,"NBTY Global, Inc.",1,
2,Sep 8 2022 12:15PM,245964,22,"NBTY Global, Inc.",1,
3,Sep 8 2022 12:11PM,245963,10,ISDIN Corporation,12,
4,Sep 8 2022 12:11PM,245959,10,ISDIN Corporation,13,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 12:17PM,245966,22,"NBTY Global, Inc.",1.0,NaN
1,Sep 8 2022 12:16PM,245965,22,"NBTY Global, Inc.",1.0,NaN
2,Sep 8 2022 12:15PM,245964,22,"NBTY Global, Inc.",1.0,NaN
3,Sep 8 2022 12:11PM,245963,10,ISDIN Corporation,12.0,NaN
4,Sep 8 2022 12:11PM,245959,10,ISDIN Corporation,13.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 12:17PM,245966,22,"NBTY Global, Inc.",1.0,0.0
1,Sep 8 2022 12:16PM,245965,22,"NBTY Global, Inc.",1.0,0.0
2,Sep 8 2022 12:15PM,245964,22,"NBTY Global, Inc.",1.0,0.0
3,Sep 8 2022 12:11PM,245963,10,ISDIN Corporation,12.0,0.0
4,Sep 8 2022 12:11PM,245959,10,ISDIN Corporation,13.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3934679,150.0,14.0
12,491899,2.0,0.0
15,245941,88.0,1.0
16,245898,4.0,0.0
18,491889,0.0,2.0
19,737835,3.0,4.0
20,737737,11.0,1.0
21,1229304,5.0,0.0
22,737895,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3934679,150.0,14.0
1,12,491899,2.0,0.0
2,15,245941,88.0,1.0
3,16,245898,4.0,0.0
4,18,491889,0.0,2.0
5,19,737835,3.0,4.0
6,20,737737,11.0,1.0
7,21,1229304,5.0,0.0
8,22,737895,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,150.0,14.0
1,12,2.0,0.0
2,15,88.0,1.0
3,16,4.0,0.0
4,18,0.0,2.0
5,19,3.0,4.0
6,20,11.0,1.0
7,21,5.0,0.0
8,22,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,150.0
1,12,Released,2.0
2,15,Released,88.0
3,16,Released,4.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Executing,14.0,0.0,1.0,0.0,2.0,4.0,1.0,0.0,0.0
Released,150.0,2.0,88.0,4.0,0.0,3.0,11.0,5.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Executing,14.0,0.0,1.0,0.0,2.0,4.0,1.0,0.0,0.0
1,Released,150.0,2.0,88.0,4.0,0.0,3.0,11.0,5.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Executing,14.0,0.0,1.0,0.0,2.0,4.0,1.0,0.0,0.0
1,Released,150.0,2.0,88.0,4.0,0.0,3.0,11.0,5.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()